In [107]:
pip install pulp

In [108]:
# Package
import pandas as pd
import numpy as np
from pulp import *

In [109]:
data = pd.read_excel('https://raw.githubusercontent.com/ikanx101/Data-Science-101/master/Competition/Tahap%20I/20201207%20Product%20list.xlsx')

## Informasi Data

---

cost_benefit = profit \\
burn_3m = budget promosi \\
product code = jenis product

In [110]:
data.head()

,product code,brand,original_sales_catalog,offer_price_today,first_price,first_order,first_order_3m,first_gmv,first_gmv_3m,discount,second_price,second_order,second_order_3m,second_gmv,second_gmv_3m,diff_order,diff_order_3m,diff_gmv,diff_gmv_3m,burn,burn_3m,elasticity,discount_bin,final_price,reguler_commission,first_profit,second_profit,increase_profit,cost_benefit
0,6000227-0001,Indomie,Consumer Goods,85000,85000,7,21,595000,1785000,1,84150.0,9,27,757350.0,2272050.0,2,6,162350.0,487050.0,7650.0,22950.0,0.002353,a_1%,84150,0.03,53550.0,68161.5,14611.5,-8338.5
1,6000094-0002,Cap Badak,Consumer Goods,4000,4000,1,3,4000,12000,1,3960.0,2,6,7920.0,23760.0,1,3,3920.0,11760.0,80.0,240.0,0.025000,a_1%,3960,0.03,360.0,712.8,352.8,112.8
2,6000100-0003,BLUE BAND,Consumer Goods,335000,335000,2,6,670000,2010000,1,331650.0,7,21,2321550.0,6964650.0,5,15,1651550.0,4954650.0,23450.0,70350.0,0.001493,a_1%,331650,0.03,60300.0,208939.5,148639.5,78289.5
3,6000301-0004,2AA-Sembako,Consumer Goods,85000,85000,3,9,255000,765000,1,84150.0,6,18,504900.0,1514700.0,3,9,249900.0,749700.0,5100.0,15300.0,0.003529,a_1%,84150,0.03,22950.0,45441.0,22491.0,7191.0
4,6000307-0005,2AA-Sembako,Consumer Goods,18000,18000,2,6,36000,108000,1,17820.0,5,15,89100.0,267300.0,3,9,53100.0,159300.0,900.0,2700.0,0.016667,a_1%,17820,0.03,3240.0,8019.0,4779.0,2079.0


In [111]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7617 entries, 0 to 7616
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   product code            7617 non-null   object 
 1   brand                   7617 non-null   object 
 2   original_sales_catalog  7617 non-null   object 
 3   offer_price_today       7617 non-null   int64  
 4   first_price             7617 non-null   int64  
 5   first_order             7617 non-null   int64  
 6   first_order_3m          7617 non-null   int64  
 7   first_gmv               7617 non-null   int64  
 8   first_gmv_3m            7617 non-null   int64  
 9   discount                7617 non-null   int64  
 10  second_price            7617 non-null   float64
 11  second_order            7617 non-null   int64  
 12  second_order_3m         7617 non-null   int64  
 13  second_gmv              7617 non-null   float64
 14  second_gmv_3m           7617 non-null   

Data sudah bersih

## Analisis Product dan Brand

In [112]:
print("Product dengan budget promosi paling mahal :\n \n" ,data[["product code", 
                                                                 "brand", "burn_3m", "cost_benefit"]].sort_values(['burn_3m'], ascending = False))

print("Product dengan profit paling kecil :\n \n" ,data[["product code", 
                                                          "brand", "burn_3m", "cost_benefit"]].sort_values(['cost_benefit'], ascending = True))

Product dengan budget promosi paling mahal :
 
       product code           brand       burn_3m  cost_benefit
5625  6000654-5626         Samsung  1.686726e+08 -1.470963e+08
4947  6000381-4948          Xiaomi  6.885864e+07 -5.558045e+07
5623  6000461-5624         Samsung  3.700458e+07 -3.215405e+07
4986  5001141-4987         Samsung  3.294063e+07 -2.359216e+07
4946  6000369-4947          Xiaomi  3.235320e+07 -2.973986e+07
...            ...             ...           ...           ...
4825  2600515-4826  FRES & NATURAL  2.160000e+02 -2.160000e+02
539   6000419-0540            DAAI  2.040000e+02  9.588000e+01
2504  6000111-2505       TEH PUCUK  1.920000e+02  9.024000e+01
4087  6000279-4088          Nabati  1.798200e+02 -5.394600e+00
2668  6001135-2669         Indomie  1.467000e+02  6.894900e+01

[7617 rows x 4 columns]
Product dengan profit paling kecil :
 
       product code             brand      burn_3m  cost_benefit
5625  6000654-5626           Samsung  168672600.0 -1.470963e+08
494

Ternyata produk yang menghabiskan banyak budget promosi justru merugikan \\
terlihat dari cost_benefit yang negatif. \\

Product dari Brand Samsung menghabiskan budget promosi paling banyak, namun justru paling merugikan.

In [113]:
order = pd.DataFrame(data.groupby('brand').sum()[['burn_3m', 'cost_benefit']])

order.sort_values(['cost_benefit'], ascending=False).head(10)

,burn_3m,cost_benefit
brand,,
Cussons Baby,16121354.25,3.130525e+07
Kapal Api,43150606.50,2.604682e+07
MARLBORO,7176810.00,1.191330e+07
Djarum Coklat,6229680.00,9.566260e+06
Sampoerna,6821790.00,8.379252e+06
Gudang Garam,11342106.00,7.689474e+06
DJI SAM SOE,13089541.50,7.246590e+06
Cordoba,6338250.00,6.239902e+06
Ngawiti Mas,4621650.00,6.174250e+06


Brand paling profit, yaitu Cussons Baby

In [114]:
order = pd.DataFrame(data.groupby('brand').sum()[['burn_3m', 'cost_benefit']])

order.sort_values(['cost_benefit'], ascending=False).tail(10)

,burn_3m,cost_benefit
brand,,
Infinix,15830730.0,-1.034312e+07
Apple,23263800.0,-1.304830e+07
Antam,21409680.6,-1.572050e+07
Al Qolam,19593600.0,-1.800431e+07
Baby Happy,30509107.8,-1.840254e+07
Vespa,22500000.0,-2.115000e+07
SGM,37979801.7,-2.460851e+07
Peugeot Scooters,26460000.0,-2.635920e+07
Xiaomi,235644211.5,-1.630430e+08


 Brand paling tidak profit adalah samsung.

Setelah melihat 10 data teratas dan terbawah berdasarkan profitnya. Kita lihat bahwa brand hp (Samsung, Xiaomi, Apple, Infinix) ternyata menghasilkan cost_benefit yang negatif. Dalam kata lain, tidak mendatangkan profit.

Sedangkan, brand yang productnya merupakan product yang mudah dibeli / product sehari-hari (Cussons Baby, Kapal Api, dsb) justru mendatangkan profit lebih besar.

Berdasarkan temuan tadi, kita ingin memberikan promosi pada product \\
dengan budget promosi yang kurang dari 200 juta sehingga product - product tersebut menghasilkan profit (cost_benefit) yang maksimal.

Selain itu, kita hanya akan memilih product dengan profit (cost_benefit) yang positif.

## Optimalisasi Kombinasi Produk ( product code ) Untuk Memaksimalkan Profit

---

Berdasarkan tujuan, kita akan menggunakan
Fungsi Tujuan : \\
$Z(C) = XC$ \\
Dengan X adalah matriks cost_benefit ukuran nx1 dan C adalah matriks ukuran 1xn \\

Konstrain : \\
burn_3m$\times$C ≤ 200juta \\
C binary ( 0 atau 1 ). \\
0 berarti tidak diberikan promosi \\
1 berarti diberikan promosi

## Produk dengan cost benefit positif

In [115]:
data_pos = data[data["cost_benefit"] > 0]

In [116]:
data_pos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1953 entries, 1 to 7616
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   product code            1953 non-null   object 
 1   brand                   1953 non-null   object 
 2   original_sales_catalog  1953 non-null   object 
 3   offer_price_today       1953 non-null   int64  
 4   first_price             1953 non-null   int64  
 5   first_order             1953 non-null   int64  
 6   first_order_3m          1953 non-null   int64  
 7   first_gmv               1953 non-null   int64  
 8   first_gmv_3m            1953 non-null   int64  
 9   discount                1953 non-null   int64  
 10  second_price            1953 non-null   float64
 11  second_order            1953 non-null   int64  
 12  second_order_3m         1953 non-null   int64  
 13  second_gmv              1953 non-null   float64
 14  second_gmv_3m           1953 non-null   

In [117]:
product = list(data_pos['product code'])

profit = dict(zip(product, data_pos['cost_benefit']))
budget = dict(zip(product, data_pos['burn_3m']))

In [118]:
# Declare Problem
prob = LpProblem("Mixed Problem", LpMaximize)

/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [119]:
product_vars = LpVariable.dicts("Product", product, cat='Binary')

In [120]:
# Obj Function
prob += lpSum([profit[i]*product_vars[i] for i in product])
# Constraint
prob += lpSum([budget[f] * product_vars[f] for f in product]) <= 200000000
prob += lpSum([product_vars[f] for f in product]) >= 1601

In [121]:
# The status of the solution is printed to the screen
prob.solve()
print("Status:", LpStatus[prob.status])

Status: Optimal


In [122]:
product_list = []
product_value = []

for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)
        product_list.append(v.name)
        product_value.append(v.varValue)

obj = value(prob.objective)
print("The total profit of combination product is: {}".format(round(obj,10)))

Product_0001941_5561 = 1.0
Product_1000070_7550 = 1.0
Product_1000140_7335 = 1.0
Product_1001000_5754 = 1.0
Product_1001079_5756 = 1.0
Product_1001124_6090 = 1.0
Product_1001316_5566 = 1.0
Product_1100253_6114 = 1.0
Product_1200148_2122 = 1.0
Product_1200682_7153 = 1.0
Product_1200885_7154 = 1.0
Product_1201512_7156 = 1.0
Product_1400250_6844 = 1.0
Product_1400542_6846 = 1.0
Product_1401737_7543 = 1.0
Product_1401803_7024 = 1.0
Product_1500067_5670 = 1.0
Product_1500202_5137 = 1.0
Product_1500850_0611 = 1.0
Product_1501028_0612 = 1.0
Product_1502867_6483 = 1.0
Product_1504328_6486 = 1.0
Product_1504352_6487 = 1.0
Product_1504353_6488 = 1.0
Product_1504353_6489 = 1.0
Product_1504429_6492 = 1.0
Product_1504433_5094 = 1.0
Product_1508811_5972 = 1.0
Product_1509670_5978 = 1.0
Product_1512716_5981 = 1.0
Product_1517021_5995 = 1.0
Product_1517041_5997 = 1.0
Product_1519128_6495 = 1.0
Product_1600240_5184 = 1.0
Product_1601354_6109 = 1.0
Product_1601723_6779 = 1.0
Product_1601899_6780 = 1.0
P

In [123]:
len(product_list)

1601

In [124]:
rename = [item.replace('_', '-') for item in product_list]
rename = [item.replace('Product-', '') for item in rename]

filter1 = data_pos["product code"].isin(rename) 
check = data_pos[filter1]

print("profit: ", check['cost_benefit'].sum())
print("Budget: ", check['burn_3m'].sum())
print("Produk: ", check['product code'].count())

profit:  205709825.8869 

Budget:  199999997.34
Produk:  1601


In [125]:
df = pd.DataFrame({'product.code' : rename,
                   'nilai' : product_value},
                  columns = ['product.code', 'nilai'])
df

,product.code,nilai
0,0001941-5561,1.0
1,1000070-7550,1.0
2,1000140-7335,1.0
3,1001000-5754,1.0
4,1001079-5756,1.0
...,...,...
1596,7000012-1711,1.0
1597,7000016-3658,1.0
1598,7000018-1692,1.0
1599,7000025-3236,1.0


In [126]:
df.to_csv("Product.csv")

# Kesimpulan

---

Dengan memberikan promosi ke 1601 produk diperoleh profit 205709825.8869 dan budget promosinya yaitu 199999997.34